In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'subset_data/train'
validation_data_dir = 'subset_data/test'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [6]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [7]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 12 images belonging to 2 classes.


In [8]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 12 images belonging to 2 classes.


In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Epoch 1/50
125/125 [==============================] - 36s - loss: 0.2369 - acc: 0.9100 - val_loss: 0.2030 - val_acc: 0.9167
Epoch 2/50
125/125 [==============================] - 35s - loss: 0.0393 - acc: 0.9893 - val_loss: 0.0968 - val_acc: 0.9167
Epoch 3/50
125/125 [==============================] - 35s - loss: 0.0499 - acc: 0.9927 - val_loss: 0.2041 - val_acc: 0.9167
Epoch 4/50
125/125 [==============================] - 32s - loss: 0.0368 - acc: 0.9947 - val_loss: 0.4705 - val_acc: 0.8333
Epoch 5/50
125/125 [==============================] - 32s - loss: 0.0428 - acc: 0.9933 - val_loss: 0.1479 - val_acc: 0.9167
Epoch 6/50
125/125 [==============================] - 32s - loss: 0.0051 - acc: 0.9993 - val_loss: 0.2189 - val_acc: 0.9167
Epoch 7/50
125/125 [==============================] - 32s - loss: 0.0529 - acc: 0.9960 - val_loss: 0.3793 - val_acc: 0.9167
Epoch 8/50
125/125 [==============================] - 32s - loss: 0.0409 - acc: 0.9960 - val_loss: 0.6327 - val_acc: 0.9167
Epoch 9/

# Bottle neck approach


leverage network pre-trained. Try with VGG16 architecture. 

In [14]:
from keras import applications
import numpy as np
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'subset_data/train'
validation_data_dir = 'subset_data/test'
nb_train_samples = 12
nb_validation_samples = 12
epochs = 25
batch_size = 2

In [15]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)


In [16]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [17]:
save_bottlebeck_features()
train_top_model()

Found 12 images belonging to 2 classes.
Found 12 images belonging to 2 classes.
Train on 12 samples, validate on 12 samples
Epoch 1/25
12/12 [==============================] - 0s - loss: 2.3867 - acc: 0.4167 - val_loss: 4.3511 - val_acc: 0.5000
Epoch 2/25
12/12 [==============================] - 0s - loss: 0.7543 - acc: 0.9167 - val_loss: 0.3924 - val_acc: 0.7500
Epoch 3/25
12/12 [==============================] - 0s - loss: 0.5200 - acc: 0.8333 - val_loss: 0.3599 - val_acc: 0.8333
Epoch 4/25
12/12 [==============================] - 0s - loss: 0.4236 - acc: 0.8333 - val_loss: 0.4792 - val_acc: 0.7500
Epoch 5/25
12/12 [==============================] - 0s - loss: 0.0317 - acc: 1.0000 - val_loss: 0.3533 - val_acc: 0.8333
Epoch 6/25
12/12 [==============================] - 0s - loss: 0.0044 - acc: 1.0000 - val_loss: 0.3561 - val_acc: 0.8333
Epoch 7/25
12/12 [==============================] - 0s - loss: 0.1776 - acc: 0.9167 - val_loss: 2.6792 - val_acc: 0.5000
Epoch 8/25
12/12 [===========